In [637]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 200
pd.options.display.max_rows = 5

# ONISEP - Higher Education Structures Dataset

1. Loading the dataset

In [638]:
onisep_dataset = pd.read_csv("./data_source/raw/onisep-ideo-structures_denseignement_superieur.csv", sep=";", low_memory=False)
onisep_dataset

,code UAI,n° SIRET,type d'établissement,nom,sigle,statut,tutelle,université de rattachement libellé,université de rattachement ID et URL Onisep,établissements liés libellés,établissements liés URL et ID Onisep,boîte postale,adresse,CP,commune,commune (COG),cedex,téléphone,arrondissement,département,académie,région,région (COG),longitude (X),latitude (Y),journées portes ouvertes,label génération 2024,URL et ID Onisep
0,0381516S,1.300214e+13,Institut universitaire de technologie,IUT 1 (site Grenoble - Gambetta),IUT 1,public,"Ministère de l'Enseignement supérieur, de la R...",Université Grenoble Alpes,www.onisep.fr/http/redirection/etablissement/s...,IUT 1 (campus - Saint-Martin-d'Hères) à Saint-...,www.onisep.fr/http/redirection/etablissement/s...,NaN,39-41 boulevard Gambetta,38000,Grenoble,38185,NaN,04 56 52 02 02,NaN,38 - Isère,Grenoble,Auvergne-Rhône-Alpes,84,5.725534,45.184863,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...
1,0270026G,1.927003e+13,"Lycée général, technologique ou polyvalent",Lycée Louise Michel,NaN,public,Ministère chargé de l'Éducation nationale et d...,NaN,NaN,NaN,NaN,NaN,23 rue d'Eragny,27140,Gisors,27284,NaN,02 32 27 69 80,NaN,27 - Eure,Normandie,Normandie,28,1.779960,49.286000,le 16/03/2024 de 09h00 à 12h00,Non,www.onisep.fr/http/redirection/etablissement/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,0875078X,1.987067e+13,Institut national supérieur du professorat et ...,Institut national supérieur du professorat et ...,INSPÉ,public,"Ministère de l'Enseignement supérieur, de la R...",Université de Limoges,www.onisep.fr/http/redirection/etablissement/s...,Institut national supérieur du professorat et ...,www.onisep.fr/http/redirection/etablissement/s...,NaN,"209 boulevard de Vanteaux, Campus Condorcet",87036,Limoges,87085,Cedex,05 55 01 76 86,NaN,87 - Haute-Vienne,Limoges,Nouvelle-Aquitaine,75,1.226521,45.822932,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...
8695,0860774Z,3.007263e+13,"Lycée général, technologique ou polyvalent",Lycée polyvalent privé Saint-Jacques de Compos...,NaN,privé sous contrat,Ministère chargé de l'Éducation nationale et d...,NaN,NaN,Centre de formation Saint-Jacques de Compostel...,www.onisep.fr/http/redirection/etablissement/s...,NaN,2 avenue de la Révolution,86036,Poitiers,86194,Cedex,05 49 61 60 60,NaN,86 - Vienne,Poitiers,Nouvelle-Aquitaine,75,0.382796,46.585900,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...


2. Replace the multiple existing terms used to define  private institution status to a unique one. As such we obtain a binary categorization of a private and public status.

In [639]:
print(onisep_dataset["statut"].unique())
onisep_dataset['statut'].replace(["privé", "privé reconnu par l'Etat", "privé sous contrat", "privé hors contrat"], 'private', inplace=True)
print(onisep_dataset["statut"].unique())
onisep_dataset = onisep_dataset.loc[onisep_dataset['statut'].notnull()]

['public' 'privé' "privé reconnu par l'Etat" 'privé sous contrat'
 'privé hors contrat']
['public' 'private']


3. Only keep major secondary education institutions (Universities, Technical Universities, Business Schools, Engineering Schools, Political Science Institutes, Polytechnic Institutes)

In [640]:
onisep_dataset = onisep_dataset[~onisep_dataset["type d'établissement"].isin([
       'Lycée général, technologique ou polyvalent',
       "Centre de formation d'apprentis", 'Lycée professionnel',
       'École du secteur social', 'Unité de formation et de recherche',
       'Lycée agricole', "Institut national supérieur du professorat et de l'éducation",
       'Conservatoire régional',
       "Service d'éducation spéciale et de soins à domicile", 'Centre de formation de fonctionnaires',
       'CREPS', 'Campus connecté','École de santé', 'Établissement national de sport',
       'Enseignement public à distance','École de formation sportive',
       "Établissement régional d'enseignement adapté",
       'Conservatoire départemental', 'Conservatoire national', "Autre établissement d'enseignement",
       'Maison familiale rurale', 'Centre de formation professionnelle'])]
print(onisep_dataset["type d'établissement"].unique())
onisep_dataset

['Institut universitaire de technologie' 'Université'
 'Grande école de commerce' "École d'art" "École d'ingénieurs"
 'École de gestion et de commerce' 'École des beaux-arts'
 "Institut d'études politiques" "École d'architecture"
 'Institut national polytechnique' 'École vétérinaire']


,code UAI,n° SIRET,type d'établissement,nom,sigle,statut,tutelle,université de rattachement libellé,université de rattachement ID et URL Onisep,établissements liés libellés,établissements liés URL et ID Onisep,boîte postale,adresse,CP,commune,commune (COG),cedex,téléphone,arrondissement,département,académie,région,région (COG),longitude (X),latitude (Y),journées portes ouvertes,label génération 2024,URL et ID Onisep
0,0381516S,1.300214e+13,Institut universitaire de technologie,IUT 1 (site Grenoble - Gambetta),IUT 1,public,"Ministère de l'Enseignement supérieur, de la R...",Université Grenoble Alpes,www.onisep.fr/http/redirection/etablissement/s...,IUT 1 (campus - Saint-Martin-d'Hères) à Saint-...,www.onisep.fr/http/redirection/etablissement/s...,NaN,39-41 boulevard Gambetta,38000,Grenoble,38185,NaN,04 56 52 02 02,NaN,38 - Isère,Grenoble,Auvergne-Rhône-Alpes,84,5.725534,45.184863,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...
4,0062211W,1.300257e+13,Institut universitaire de technologie,IUT Nice Côte d'Azur - Site de Cannes,NaN,public,"Ministère de l'Enseignement supérieur, de la R...",Université Côte d'Azur,www.onisep.fr/http/redirection/etablissement/s...,IUT Nice Côte d'Azur - Site de Menton à Menton...,www.onisep.fr/http/redirection/etablissement/s...,NaN,"214 avenue Francis Tonner, Campus Georges Méliès",6150,Cannes,06029,NaN,04 89 15 33 02,NaN,06 - Alpes-Maritimes,Nice,Provence-Alpes-Côte d'Azur,93,6.959951,43.550170,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,0311381H,1.931138e+13,Institut national polytechnique,Toulouse INP,Toulouse INP,public,"Ministère de l'Enseignement supérieur, de la R...",NaN,NaN,NaN,NaN,BP 34038,6 allée Emile Monso,31029,Toulouse,31555,Cedex 04,05 34 32 30 00,NaN,31 - Haute-Garonne,Toulouse,Occitanie,76,1.454305,43.602716,NaN,Oui,www.onisep.fr/http/redirection/etablissement/s...
8693,0134334R,8.194642e+13,École de gestion et de commerce,My BS - My Business School l Aix-en-Provence,NaN,private,NaN,NaN,NaN,NaN,NaN,NaN,"75 rue Marcellin Berthelot, Pôle d'Activité d'...",13290,Aix-en-Provence,13001,NaN,04.84.32.63.26,NaN,13 - Bouches-du-Rhône,Aix-Marseille,Provence-Alpes-Côte d'Azur,93,5.388992,43.272279,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...


4. Only keep the main structures, mother insititutions

In [641]:
onisep_dataset = onisep_dataset.loc[onisep_dataset['université de rattachement libellé'].isnull()]
onisep_dataset

,code UAI,n° SIRET,type d'établissement,nom,sigle,statut,tutelle,université de rattachement libellé,université de rattachement ID et URL Onisep,établissements liés libellés,établissements liés URL et ID Onisep,boîte postale,adresse,CP,commune,commune (COG),cedex,téléphone,arrondissement,département,académie,région,région (COG),longitude (X),latitude (Y),journées portes ouvertes,label génération 2024,URL et ID Onisep
5,0062205P,1.300257e+13,Université,Université Côte d'Azur,UniCA,public,"Ministère de l'Enseignement supérieur, de la R...",NaN,NaN,NaN,NaN,BP 2135,"28 avenue de Valrose, Grand Château Parc Valrose",6103,Nice,06088,Cedex 02,04 92 07 60 60,NaN,06 - Alpes-Maritimes,Nice,Provence-Alpes-Côte d'Azur,93,7.265890,43.716760,NaN,Oui,www.onisep.fr/http/redirection/etablissement/s...
24,0755956S,8.342954e+13,Grande école de commerce,Neoma Business School,NEOMA BS,private,"Ministère de l'Enseignement supérieur, de la R...",NaN,NaN,CFA Reims Management School à Reims (51) | Neo...,www.onisep.fr/http/redirection/etablissement/s...,NaN,6 rue Vandrezanne,75013,Paris,75100,NaN,01 73 06 98 00,13.0,75 - Paris,Paris,Ile-de-France,11,2.355393,48.828113,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8692,0311381H,1.931138e+13,Institut national polytechnique,Toulouse INP,Toulouse INP,public,"Ministère de l'Enseignement supérieur, de la R...",NaN,NaN,NaN,NaN,BP 34038,6 allée Emile Monso,31029,Toulouse,31555,Cedex 04,05 34 32 30 00,NaN,31 - Haute-Garonne,Toulouse,Occitanie,76,1.454305,43.602716,NaN,Oui,www.onisep.fr/http/redirection/etablissement/s...
8693,0134334R,8.194642e+13,École de gestion et de commerce,My BS - My Business School l Aix-en-Provence,NaN,private,NaN,NaN,NaN,NaN,NaN,NaN,"75 rue Marcellin Berthelot, Pôle d'Activité d'...",13290,Aix-en-Provence,13001,NaN,04.84.32.63.26,NaN,13 - Bouches-du-Rhône,Aix-Marseille,Provence-Alpes-Côte d'Azur,93,5.388992,43.272279,NaN,Non,www.onisep.fr/http/redirection/etablissement/s...


5. Only keep features of interest

In [642]:
print(onisep_dataset["type d'établissement"].unique())
onisep_dataset = onisep_dataset[["code UAI", "nom", "statut","région"]]
onisep_dataset['web'] = np.nan
onisep_dataset

['Université' 'Grande école de commerce' "École d'art"
 "École d'ingénieurs" 'École de gestion et de commerce'
 'École des beaux-arts' "École d'architecture"
 'Institut national polytechnique' 'École vétérinaire'
 "Institut d'études politiques"]


/tmp/ipykernel_11046/4284492174.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onisep_dataset['web'] = np.nan


,code UAI,nom,statut,région,web
5,0062205P,Université Côte d'Azur,public,Provence-Alpes-Côte d'Azur,NaN
24,0755956S,Neoma Business School,private,Ile-de-France,NaN
...,...,...,...,...,...
8692,0311381H,Toulouse INP,public,Occitanie,NaN
8693,0134334R,My BS - My Business School l Aix-en-Provence,private,Provence-Alpes-Côte d'Azur,NaN


6. Select the second ONISEP dataset in order to obtain the website from all institutions.

In [643]:

# Above level 6 corresponds to Licence, licence professionnelle, BUT : BUT : Bachelor universitaire de technologie (https://www.service-public.fr/particuliers/vosdroits/F199)
dataset_containing_urls = pd.read_csv("./data_source/raw/onisep-ideo-actions-de-formation.csv")
dataset_containing_urls

,Unnamed: 0,Action de Formation (AF) identifiant Onisep,Formation (FOR) libellé,FOR URL et ID Onisep,FOR indexation domaine web Onisep,FOR type,FOR nature du certificat,FOR URL référentiel,FOR niveau de sortie,Lieu d'enseignement (ENS) libellé,ENS URL et ID Onisep,ENS code UAI,ENS statut,ENS adresse,ENS code postal,ENS commune,ENS département,ENS académie,ENS région,ENS latitude,ENS longitude,ENS n° téléphone,ENS site web,ENS hébergement,ENS accessibilité,AF durée cycle standard,AF modalités scolarité,AF éléments d'enseignement,AF coût scolarité,AF coût d'inscription,AF modalités accueil,AF page web,AF date de modification
0,0,AF.59038,DN MADE mention espace,http://www.onisep.fr/http/redirection/formatio...,"arts, culture, artisanat/arts appliqués| arts,...",diplôme national des métiers d'art et du design,Diplôme national ou diplôme d'État,NaN,Bac + 3,Lycée polyvalent Joseph Vallot,www.onisep.fr/http/redirection/etablissement/s...,0340028W,public,rue du docteur Henri Mas,34700,Lodève,Hérault,Montpellier,Occitanie,43.7310,3.31485,04 67 88 48 60,https://joseph-vallot.mon-ent-occitanie.fr/,internat (homme/femme) : pour les sections pro...,Locaux accessibles : Accessibilité totale pour...,3 ans,"temps plein, cours en présentiel","cadre de vie : matériaux, patrimoine et enviro...",NaN,NaN,NaN,NaN,15/03/2023
1,1,AF.24272,BTS Électrotechnique,http://www.onisep.fr/http/redirection/formatio...,"mécanique/automatismes| électricité, électroni...",brevet de technicien supérieur,Diplôme national ou diplôme d'État,http://eduscol.education.fr/sti/sites/eduscol....,Bac + 2,Lycée l'Essouriau,www.onisep.fr/http/redirection/etablissement/s...,0911492C,public,"1 avenue de Dordogne, BP 1031",91979,Les Ulis,Essonne,Versailles,Ile-de-France,48.6761,2.16504,01 64 86 82 82,http://www.lyc-lessouriau-les-ulis.ac-versaill...,sans hébergement,NaN,2 ans,"temps plein, cours en présentiel",NaN,NaN,NaN,NaN,NaN,06/06/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28252,28252,AF.126015,MC (futur CS) Services numériques aux organisa...,http://www.onisep.fr/http/redirection/formatio...,"informatique, Internet/développement, programm...",mention complémentaire (futur CS),Diplôme national ou diplôme d'État,https://eduscol.education.fr/referentiels-prof...,Bac + 1,Lycée professionnel René Cassin,www.onisep.fr/http/redirection/etablissement/s...,0420029D,public,11 chemin Sainte Barbe,42800,Rive-de-Gier,Loire,Lyon,Auvergne-Rhône-Alpes,45.5211,4.60543,04 77 75 03 44,https://lp-rene-cassin-rive-de-gier.ent.auverg...,sans hébergement,Locaux accessibles : Accessibilité totale pour...,1 an,"temps plein, cours en présentiel",NaN,NaN,NaN,NaN,NaN,23/01/2024
28253,28253,AF.9915,FCIL Électricien du spectacle,http://www.onisep.fr/http/redirection/formatio...,"arts, culture, artisanat/arts du spectacle| él...",formation complémentaire d'initiative locale,Formation d'initiative locale,NaN,Bac + 1,Lycée polyvalent Jules Verne,www.onisep.fr/http/redirection/etablissement/s...,0783431F,public,"2 rue de la Constituante, CS 80238",78500,Sartrouville,Yvelines,Versailles,Ile-de-France,48.9431,2.15947,01 61 04 13 00,http://www.lyc-verne-sartrouville.ac-versaille...,sans hébergement,NaN,1 an,"temps plein, cours en présentiel",NaN,NaN,NaN,NaN,NaN,15/03/2023


In [644]:
for index, row in onisep_dataset.iterrows():
    uai_code = row["code UAI"]
    corresponding_web_site = dataset_containing_urls.loc[dataset_containing_urls["ENS code UAI"] == uai_code]
    corresponding_web_site_value = corresponding_web_site.head(1)['ENS site web'].values
    if corresponding_web_site_value:
        onisep_dataset.at[index,'web'] = corresponding_web_site_value[0]
    
onisep_dataset = onisep_dataset.loc[onisep_dataset["web"].notna()]
onisep_dataset

/tmp/ipykernel_11046/594126845.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if corresponding_web_site_value:


,code UAI,nom,statut,région,web
24,0755956S,Neoma Business School,private,Ile-de-France,https://www.neoma-bs.fr
27,0755958U,1984 School Of Design,private,Ile-de-France,http://www.1984-ecole.com
...,...,...,...,...,...
8682,0331877L,École supérieure des Beaux-Arts de Bordeaux,public,Nouvelle-Aquitaine,https://www.ebabx.fr
8693,0134334R,My BS - My Business School l Aix-en-Provence,private,Provence-Alpes-Côte d'Azur,http://www.my-bs.fr


# Ministry of Higher Education & Research - Secondary Higher Education Annuary Dataset

In [645]:
second_dataset = pd.read_csv("./data_source/raw/annuaire-education-ministere-recherche.csv", sep=";", low_memory=False)
second_dataset

,etablissement_id_paysage,uo_lib,nom_court,sigle,type_d_etablissement,typologie_d_universites_et_assimiles,secteur_d_etablissement,vague_contractuelle,localisation,url,coordonnees,uai,anciens_codes_uai,siret,siren,rna,identifiant_wikidata,element_wikidata,identifiant_idref,identifiant_eter,identifiant_ror,element_ror,identifiant_pic,identifiant_orgref,identifiant_isni,element_isni,element_fundref,date_creation,texte_de_ref_creation_lib,texte_de_ref_creation,com_code,com_nom,uucr_id,uucr_nom,dep_id,dep_nom,aca_id,aca_nom,reg_id,reg_nom,reg_id_old,reg_nom_old,mention_distribution,adresse_uai,lieu_dit_uai,boite_postale_uai,code_postal_uai,localite_acheminement_uai,pays_etranger_acheminement,numero_telephone_uai,statut_juridique_court,statut_juridique_long,compte_facebook,compte_twitter,compte_instagram,compte_flickr,compte_pinterest,flux_rss,compte_linkedin,compte_france_culture,compte_scribd,compte_scoopit,compte_tumblr,compte_youtube,compte_vimeo,compte_dailymotion,compte_github,wikipedia,wikipedia_en,scanr,hal,mooc,article,uo_lib_officiel,uo_lib_en,url_en,url_cn,url_es,url_de,url_it,universites_fusionnees,etablissement_experimental,champ_recherche,inscrits,inscrits_2010,inscrits_2011,inscrits_2012,inscrits_2013,inscrits_2014,inscrits_2015,inscrits_2016,inscrits_2017,inscrits_2018,inscrits_2019,inscrits_2020,inscrits_2021,inscrits_2022
0,fWJJA,Centrale Lyon,EC Lyon,NaN,École,NaN,public,Vague A,Auvergne-Rhône-Alpes>Lyon>Rhône>Lyon>Écully,https://www.ec-lyon.fr/,"45.78299,4.76927",0690187D,NaN,19690187000010,196901870,NaN,Q10177,https://www.wikidata.org/entity/Q10177,033894221,FR0153,05s6rge65,https://ror.org/05s6rge65,NaN,83157.0,0000 0001 2181 0799,http://www.isni.org/0000 0001 2181 0799,NaN,1857,NaN,NaN,69081,Écully,UU00760,Lyon,D069,Rhône,A10,Lyon,R84,Auvergne-Rhône-Alpes,R82,Rhône-Alpes,NaN,36 avenue Guy de Collongue,NaN,NaN,69134.0,Écully CEDEX,France,3.347219e+10,EPSCP,"Établissement public à caractère scientifique,...",https://www.facebook.com/CentraleLyon,https://twitter.com/CentraleLyon,https://www.instagram.com/centralelyon/,NaN,NaN,http://www.ec-lyon.fr/taxonomy/term/1%2B2%2B3%...,https://www.linkedin.com/school/ecole-centrale...,NaN,NaN,NaN,NaN,https://www.youtube.com/user/EcoleCentraleLyon,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/%C3%89cole_centr...,https://en.wikipedia.org/wiki/%C3%89cole_centr...,https://scanr.enseignementsup-recherche.gouv.f...,NaN,NaN,à l',École centrale de Lyon,NaN,https://www.ec-lyon.fr/en,https://www.ec-lyon.fr/zh,https://www.ec-lyon.fr/es,NaN,NaN,NaN,NaN,École centrale lyonnaise pour l'industrie et l...,https://data.enseignementsup-recherche.gouv.fr...,1482.0,1390.0,1513.0,1633.0,1714.0,1774.0,1817.0,1907.0,1876.0,1842.0,2912.0,2916.0,2920.0
1,1tI7C,Centrale Méditerranée,NaN,NaN,École,NaN,public,Vague C,Provence-Alpes-Côte d'Azur>Aix-Marseille>Bouch...,https://www.centrale-mediterranee.fr/fr,"43.34136,5.43829",0133774G,0132396J;0133401B;0130219T,19133340000015,191333400,NaN,Q273454,https://www.wikidata.org/entity/Q273454,163078998,FR0006,040baw385,https://ror.org/040baw385,NaN,7756351.0,0000 0004 1762 2703,http://www.isni.org/0000 0004 1762 2703,NaN,2003-10-02,Décret n° 2003-929 du 29 septembre 2003 portan...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,13213,Marseille 13e,UU00759,Marseille-Aix-en-Provence,D013,Bouches-du-Rhône,A02,Aix-Marseille,R93,Provence-Alpes-Côte d'Azur,R93,Provence-Alpes-Côte d'Azur,Pôle de l'Étoile - Technopôle de Château-Gombert,38 rue Frédéric-Joliot-Curie,NaN,NaN,13451.0,Marseille CEDEX 13,France,3.349128e+10,EPSCP,"Établissement public à caractère scientifique,...",https://www.facebook.com/CentraleMediterranee,https://twitter.com/Centrale_Med,https://www.instagram.com/centralemed/,NaN,NaN,NaN,https://www.linkedin.com/school/centralemedite...,NaN,NaN,NaN,NaN,https://www.youtube.com/user/CentraleMarseille,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Centrale_M%C3%A9...,https://en.wikipedia.org/wiki/%C3%89cole_centr...,https://scanr.enseignementsup-recherche.gouv

In [646]:
# Remove foreign High Schools
second_dataset.drop(index=[3,88, 129],axis=1, inplace=True)
second_dataset

,etablissement_id_paysage,uo_lib,nom_court,sigle,type_d_etablissement,typologie_d_universites_et_assimiles,secteur_d_etablissement,vague_contractuelle,localisation,url,coordonnees,uai,anciens_codes_uai,siret,siren,rna,identifiant_wikidata,element_wikidata,identifiant_idref,identifiant_eter,identifiant_ror,element_ror,identifiant_pic,identifiant_orgref,identifiant_isni,element_isni,element_fundref,date_creation,texte_de_ref_creation_lib,texte_de_ref_creation,com_code,com_nom,uucr_id,uucr_nom,dep_id,dep_nom,aca_id,aca_nom,reg_id,reg_nom,reg_id_old,reg_nom_old,mention_distribution,adresse_uai,lieu_dit_uai,boite_postale_uai,code_postal_uai,localite_acheminement_uai,pays_etranger_acheminement,numero_telephone_uai,statut_juridique_court,statut_juridique_long,compte_facebook,compte_twitter,compte_instagram,compte_flickr,compte_pinterest,flux_rss,compte_linkedin,compte_france_culture,compte_scribd,compte_scoopit,compte_tumblr,compte_youtube,compte_vimeo,compte_dailymotion,compte_github,wikipedia,wikipedia_en,scanr,hal,mooc,article,uo_lib_officiel,uo_lib_en,url_en,url_cn,url_es,url_de,url_it,universites_fusionnees,etablissement_experimental,champ_recherche,inscrits,inscrits_2010,inscrits_2011,inscrits_2012,inscrits_2013,inscrits_2014,inscrits_2015,inscrits_2016,inscrits_2017,inscrits_2018,inscrits_2019,inscrits_2020,inscrits_2021,inscrits_2022
0,fWJJA,Centrale Lyon,EC Lyon,NaN,École,NaN,public,Vague A,Auvergne-Rhône-Alpes>Lyon>Rhône>Lyon>Écully,https://www.ec-lyon.fr/,"45.78299,4.76927",0690187D,NaN,19690187000010,196901870,NaN,Q10177,https://www.wikidata.org/entity/Q10177,033894221,FR0153,05s6rge65,https://ror.org/05s6rge65,NaN,83157.0,0000 0001 2181 0799,http://www.isni.org/0000 0001 2181 0799,NaN,1857,NaN,NaN,69081,Écully,UU00760,Lyon,D069,Rhône,A10,Lyon,R84,Auvergne-Rhône-Alpes,R82,Rhône-Alpes,NaN,36 avenue Guy de Collongue,NaN,NaN,69134.0,Écully CEDEX,France,3.347219e+10,EPSCP,"Établissement public à caractère scientifique,...",https://www.facebook.com/CentraleLyon,https://twitter.com/CentraleLyon,https://www.instagram.com/centralelyon/,NaN,NaN,http://www.ec-lyon.fr/taxonomy/term/1%2B2%2B3%...,https://www.linkedin.com/school/ecole-centrale...,NaN,NaN,NaN,NaN,https://www.youtube.com/user/EcoleCentraleLyon,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/%C3%89cole_centr...,https://en.wikipedia.org/wiki/%C3%89cole_centr...,https://scanr.enseignementsup-recherche.gouv.f...,NaN,NaN,à l',École centrale de Lyon,NaN,https://www.ec-lyon.fr/en,https://www.ec-lyon.fr/zh,https://www.ec-lyon.fr/es,NaN,NaN,NaN,NaN,École centrale lyonnaise pour l'industrie et l...,https://data.enseignementsup-recherche.gouv.fr...,1482.0,1390.0,1513.0,1633.0,1714.0,1774.0,1817.0,1907.0,1876.0,1842.0,2912.0,2916.0,2920.0
1,1tI7C,Centrale Méditerranée,NaN,NaN,École,NaN,public,Vague C,Provence-Alpes-Côte d'Azur>Aix-Marseille>Bouch...,https://www.centrale-mediterranee.fr/fr,"43.34136,5.43829",0133774G,0132396J;0133401B;0130219T,19133340000015,191333400,NaN,Q273454,https://www.wikidata.org/entity/Q273454,163078998,FR0006,040baw385,https://ror.org/040baw385,NaN,7756351.0,0000 0004 1762 2703,http://www.isni.org/0000 0004 1762 2703,NaN,2003-10-02,Décret n° 2003-929 du 29 septembre 2003 portan...,https://www.legifrance.gouv.fr/jorf/id/JORFTEX...,13213,Marseille 13e,UU00759,Marseille-Aix-en-Provence,D013,Bouches-du-Rhône,A02,Aix-Marseille,R93,Provence-Alpes-Côte d'Azur,R93,Provence-Alpes-Côte d'Azur,Pôle de l'Étoile - Technopôle de Château-Gombert,38 rue Frédéric-Joliot-Curie,NaN,NaN,13451.0,Marseille CEDEX 13,France,3.349128e+10,EPSCP,"Établissement public à caractère scientifique,...",https://www.facebook.com/CentraleMediterranee,https://twitter.com/Centrale_Med,https://www.instagram.com/centralemed/,NaN,NaN,NaN,https://www.linkedin.com/school/centralemedite...,NaN,NaN,NaN,NaN,https://www.youtube.com/user/CentraleMarseille,NaN,NaN,NaN,https://fr.wikipedia.org/wiki/Centrale_M%C3%A9...,https://en.wikipedia.org/wiki/%C3%89cole_centr...,https://scanr.enseignementsup-recherche.gouv

In [647]:
# Only keep features of interest
second_dataset = second_dataset[["uai", "uo_lib", "secteur_d_etablissement","reg_nom", 'url']]
second_dataset = second_dataset.rename(columns={'uai':'code UAI',	'uo_lib':'nom',	'secteur_d_etablissement':'statut', 'reg_nom':'région', "url":'web'})
# Only keep insititutions wit a non nan inst. code
second_dataset = second_dataset[second_dataset['code UAI'].notna()]
second_dataset = second_dataset[second_dataset['web'].notna()]
second_dataset

,code UAI,nom,statut,région,web
0,0690187D,Centrale Lyon,public,Auvergne-Rhône-Alpes,https://www.ec-lyon.fr/
1,0133774G,Centrale Méditerranée,public,Provence-Alpes-Côte d'Azur,https://www.centrale-mediterranee.fr/fr
...,...,...,...,...,...
248,0931238R,Université Sorbonne Paris Nord,public,Île-de-France,http://www.univ-paris13.fr/
249,0311383K,Université Toulouse - Jean Jaurès,public,Occitanie,https://www.univ-tlse2.fr/


# Merging Datasets

1. Perform union of the two dataframes.

In [648]:
pd.options.display.max_rows = 600
df = pd.merge(second_dataset, onisep_dataset, on='code UAI',  how = 'outer')
for index, row in df.iterrows():
    if pd.isnull(row["nom_x"]): 
        df.at[index,'nom_x'] = row["nom_y"]
    if pd.isnull(row["région_x"]):
        df.at[index,'région_x'] = row["région_y"]
    if pd.isnull(row["web_x"]):
        df.at[index,'web_x'] = row["web_y"]
    if pd.isnull(row["statut_x"]):
        df.at[index,'statut_x'] = row["statut_y"]

df = df.drop(columns=['code UAI', 'nom_y', 'statut_y', 'région_y', 'web_y'])
df

,nom_x,statut_x,région_x,web_x
0,Centrale Lyon,public,Auvergne-Rhône-Alpes,https://www.ec-lyon.fr/
1,Centrale Méditerranée,public,Provence-Alpes-Côte d'Azur,https://www.centrale-mediterranee.fr/fr
2,École d'ingénieurs de Purpan,privé,Occitanie,http://www.purpan.fr/
3,École nationale supérieure de mécanique et d'a...,public,Nouvelle-Aquitaine,https://www.ensma.fr/
4,École normale supérieure de Lyon,public,Auvergne-Rhône-Alpes,http://www.ens-lyon.eu/
...,...,...,...,...
886,MJM Graphic Design,private,Occitanie,http://www.mjm-design.com/ecole/toulouse
887,Mode Estah Paris,private,Ile-de-France,http://www.mode-estah.com
888,emlyon business school,private,Ile-de-France,https://em-lyon.com/
889,École supérieure des Beaux-Arts de Bordeaux,public,Nouvelle-Aquitaine,https://www.ebabx.fr


1.b We delete all child-campuses while only keeping the mains.

In [649]:
from urllib.parse import urlparse

# Remove pending slashes to recognize duplicates
def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.scheme + '://' + parsed_url.netloc
    return domain

# Apply function to DataFrame
df['web_x'] = df['web_x'].apply(extract_domain)
df['région_x'] = df['région_x'].replace('Île-de-France', 'Ile-de-France')

# ESME's main campus is in Paris
df = df[~df['nom_x'].isin(['ECAM-EPMI - site de Grasse','ESME - campus de Lyon', 'ESME - campus de Bordeaux', "ESME - campus d'Ivry-sur-Seine", "ESME - campus de Lille"])]
df = df[~((df['web_x'] == 'https://www.ecam-epmi.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df.loc[df["nom_x"] != "Centrale Méditerranée - Campus de Nice"]
df = df[~((df['web_x'] == 'https://www.unilasalle.fr') & (df['région_x'] != 'hauts-de-France'))]
df = df.loc[df["nom_x"] != "École spéciale des travaux publics du bâtiment et de l'industrie - campus de Dijon"]
df = df[~((df['web_x'] == 'http://www.eseo.fr') & (df['région_x'] != 'Pays de la Loire'))]
df = df[~((df['web_x'] == 'https://www.esiea.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://kedge.edu') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://www.essca.fr') & (df['région_x'] != 'Pays de la Loire'))]
df = df[~((df['web_x'] == 'http://www.elisa-aerospace.fr') & (df['région_x'] != 'Hauts-de-France'))]
df = df[~((df['web_x'] == 'http://www.estaca.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://www.neoma-bs.fr') & (df['région_x'] != 'Normandie'))]
df = df[~((df['web_x'] == 'https://www.audencia.com') & (df['région_x'] != 'Pays de la Loire'))]
df = df[~((df['web_x'] == 'https://www.bsb-education.com') & (df['région_x'] != 'Bourgogne-Franche-Comté'))]
df = df[~((df['web_x'] == 'https://www.excelia-group.com') & (df['région_x'] != 'Nouvelle-Aquitaine'))]
df = df[~((df['web_x'] == 'https://www.icn-artem.com') & (df['région_x'] != 'Grand Est'))]
df = df[~((df['web_x'] == 'http://www.epf.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://www.junia.com') & (df['région_x'] != 'Hauts-de-France'))]
df = df[~((df['web_x'] == 'https://www.cesi.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://www.hear.fr') & (df['région_x'] != 'Grand Est'))]
df = df[~((df['web_x'] == 'http://www.ipacbachelorfactory.com') & (df['région_x'] != 'Pays de la Loire'))]
df = df[~((df['web_x'] == 'http://www.lepontsuperieur.eu') & (df['région_x'] != 'Pays de la Loire'))]
df = df[~((df['web_x'] == 'http://www.cohl.fr') & (df['région_x'] != 'Auvergne-Rhône-Alpes'))]
df = df[~((df['web_x'] == 'http://ecole-ingenieurs.cesi.fr') & (df['région_x'] != 'Nouvelle-Aquitaine'))]
df = df[~((df['web_x'] == 'https://www.inseec.com') & (df['région_x'] != 'Nouvelle-Aquitaine'))]
df = df[~((df['web_x'] == 'http://www.itecom-artdesign.com') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://em-lyon.com') & (df['région_x'] != 'Auvergne-Rhône-Alpes'))]
df = df[~((df['web_x'] == 'https://www.skema-bs.fr') & (df['région_x'] != "Provence-Alpes-Côte d'Azur"))]
df = df[~((df['web_x'] == 'https://www.ece.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://www.tbs-education.fr') & (df['région_x'] != 'Occitanie'))]
df = df[~((df['web_x'] == 'https://www.esigelec.fr') & (df['région_x'] != 'Normandie'))]
df = df[~((df['web_x'] == 'https://artsetmetiers.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'http://www.mines-stetienne.fr') & (df['région_x'] != 'Auvergne-Rhône-Alpes'))]
df = df[~((df['web_x'] == 'https://www.groupe-esa.com') & (df['région_x'] != 'Pays de la Loire'))]
df = df[~((df['web_x'] == 'https://www.esilv.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'http://www.icam.fr') & (df['région_x'] != 'Ile-de-France'))]
df = df[~((df['web_x'] == 'https://www.isg.fr') & (df['région_x'] != 'Ile-de-France'))]


# Find rows with same URL but different regions
df.drop_duplicates(subset=['web_x'], keep='first', inplace=True)

2. Group togetger the 'Island Regions' and remove insitutions located abroad

In [650]:
# Conform to the the standard naming
df = df.rename(columns={'région_x':'Region', 'nom_x':'Name', 'statut_x':'Category', 'web_x':'Url'})

df['Region'] = df['Region'].replace("Collectivités d'outre-mer", "Outre-mer")
df['Region'] = df['Region'].replace("Mayotte", "Outre-mer")
df['Region'] = df['Region'].replace("Martinique", "Outre-mer")
df['Region'] = df['Region'].replace("La Réunion", "Outre-mer")
df['Region'] = df['Region'].replace("Guyane", "Outre-mer")
df['Region'] = df['Region'].replace("Guadeloupe", "Outre-mer")
df = df[df['Region'] != 'Étranger']

3. Arrange and clean up the final dataset

In [651]:
# Arrange order of columns
desired_columns_order = ['Region', 'Name', 'Category', 'Url']
df = df[desired_columns_order]

# Sort Alphabetically by Region
df = df.sort_values(by="Region")
df = df.reset_index(drop=True)
df = df.rename_axis('ID')

4. Remove incoherencies by hand

In [652]:
df = df.loc[df["Name"] !="UCPA - Ecole des DJ"]
df = df.loc[df["Url"] !="https://bachelors.audencia.com"]

5. Save to csv file.

In [653]:
df.to_csv("./data_source/processed/merged-sieved-dataset.csv")